In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
coverage_identifier = "tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp26-MAM"
point_coords = "POINT(11.5469 44.9524)"
date_range = "../.."

cov = CoverageInternal(
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, coverage_identifier), 
    identifier=coverage_identifier
)

In [4]:
[c.identifier for c in operations.get_related_uncertainty_coverage_configurations(session, cov)]

['tas_seasonal_anomaly_model_ensemble_lower_uncertainty-annual-model_ensemble-tas-anomaly-rcp26-lower_bound-MAM',
 'tas_seasonal_anomaly_model_ensemble_upper_uncertainty-annual-model_ensemble-tas-anomaly-rcp26-upper_bound-MAM']

In [3]:
cov.configuration.retrieve_configuration_parameters(cov.identifier)

id_parts=['annual', 'model_ensemble', 'tas', 'anomaly', 'rcp26', 'MAM']
about to process the found pattern_parts...
checking configuration_parameter_name='aggregation_period'...
id_part='annual'
checking configuration_parameter_name='climatological_model'...
id_part='model_ensemble'
checking configuration_parameter_name='climatological_variable'...
id_part='tas'
checking configuration_parameter_name='measure'...
id_part='anomaly'
checking configuration_parameter_name='scenario'...
id_part='rcp26'
checking configuration_parameter_name='year_period'...
id_part='MAM'
result={'aggregation_period': 'annual', 'climatological_model': 'model_ensemble', 'climatological_variable': 'tas', 'measure': 'anomaly', 'scenario': 'rcp26', 'year_period': 'MAM'}


{'aggregation_period': 'annual',
 'climatological_model': 'model_ensemble',
 'climatological_variable': 'tas',
 'measure': 'anomaly',
 'scenario': 'rcp26',
 'year_period': 'MAM'}

In [4]:
cov.configuration.coverage_id_pattern

'{name}-{aggregation_period}-{climatological_model}-{climatological_variable}-{measure}-{scenario}-{year_period}'

In [6]:
db.generate_coverage_identifiers(cov.configuration)

['tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp26-DJF',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp26-MAM',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp26-JJA',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp26-SON',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp45-DJF',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp45-MAM',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp45-JJA',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp45-SON',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp85-DJF',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp85-MAM',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp85-JJA',
 'tas_seasonal_anomaly_model_ensemble-annual-model_ensemble-tas-anomaly-rcp8

In [4]:
for related_cov in operations.get_related_coverages(cov):
    print(cov.identifier, related_cov.identifier)
    print("----------")

tas_annual_absolute_model_ensemble-annual-model_ensemble-tas-absolute-rcp26-year tas_annual_absolute_model_ec_earth_cclm4_8_17-annual-ec_earth_cclm_4_8_17-tas-absolute-rcp26-year
----------
tas_annual_absolute_model_ensemble-annual-model_ensemble-tas-absolute-rcp26-year tas_annual_absolute_model_ec_earth_racmo22e-annual-ec_earth_racmo22e-tas-absolute-rcp26-year
----------
tas_annual_absolute_model_ensemble-annual-model_ensemble-tas-absolute-rcp26-year tas_annual_absolute_model_ec_earth_rca4-annual-ec_earth_rca4-tas-absolute-rcp26-year
----------
tas_annual_absolute_model_ensemble-annual-model_ensemble-tas-absolute-rcp26-year tas_annual_absolute_model_hadgem2_es_racmo22e-annual-hadgem2_racmo22e-tas-absolute-rcp26-year
----------
tas_annual_absolute_model_ensemble-annual-model_ensemble-tas-absolute-rcp26-year tas_annual_absolute_model_mpi_esm_lr_remo2009-annual-mpi_esm_lr_remo2009-tas-absolute-rcp26-year
----------
